# PyTorch Model from torchvision - Quantization for IMX500

[Run this tutorial in Google Colab](https://colab.research.google.com/github/sony/model_optimization/blob/main/tutorials/notebooks/imx500_notebooks/pytorch/pytorch_torchvision_classification_model_for_imx500.ipynb)

## Overview

In this tutorial, we will illustrate a basic and quick process of preparing a pre-trained model for deployment using MCT. 
We will use an existing pre-trained model from [torchvision](https://pytorch.org/vision/stable/models.html). The user can choose any torchvision model from this list.   

## Setup
### Install the relevant packages

In [ ]:
!pip install -q torch
!pip install -q torchvision
!pip install -q onnx

Install MCT (if it’s not already installed). Additionally, in order to use all the necessary utility functions for this tutorial, we also copy [MCT tutorials folder](https://github.com/sony/model_optimization/tree/main/tutorials) and add it to the system path.

In [ ]:
import sys
import importlib

if not importlib.util.find_spec('model_compression_toolkit'):
    !pip install model_compression_toolkit
!git clone https://github.com/sony/model_optimization.git temp_mct && mv temp_mct/tutorials . && \rm -rf temp_mct
sys.path.insert(0,"tutorials")

### Download ImageNet validation set
Download ImageNet dataset with only the validation split.

Note that for demonstration purposes we use the validation set for the model quantization routines. Usually, a subset of the training dataset is used, but loading it is a heavy procedure that is unnecessary for the sake of this demonstration.

This step may take several minutes...

In [ ]:
import os
if not os.path.isdir('imagenet'):
    !mkdir imagenet
    !wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_devkit_t12.tar.gz
    !mv ILSVRC2012_devkit_t12.tar.gz imagenet/
    !wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
    !mv ILSVRC2012_img_val.tar imagenet/

## Model Quantization

### Download a Pre-Trained Model

In [16]:
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights

model = mobilenet_v2(weights=MobileNet_V2_Weights)
model.eval()

/Vols/vol_design/tools/swat/users/yardeny/projects/model_optimization/mct_venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

### Post training quantization using Model Compression Toolkit 

Now, we're all set to use MCT's post-training quantization. To begin, we'll define a representative dataset and proceed with the model quantization. Please note that, for demonstration purposes, we'll use the evaluation dataset as our representative dataset. We'll calibrate the model using 80 representative images, divided into 20 iterations of 'batch_size' images each. 

In [17]:
import model_compression_toolkit as mct
from model_compression_toolkit.core.pytorch.pytorch_device_config import get_working_device
from typing import Iterator, Tuple, List
from torch.utils.data import DataLoader
from torchvision import transforms, datasets


BATCH_SIZE = 4
n_iters = 20
device = get_working_device()

# Define transformations for the validation set
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

# Extract ImageNet validation dataset using torchvision "datasets" module
val_dataset = datasets.ImageNet(root='./imagenet', split='val', transform=val_transform)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# Define representative dataset generator
def get_representative_dataset(n_iter: int, dataset_loader: Iterator[Tuple]):
    """
    This function creates a representative dataset generator. The generator yields numpy
        arrays of batches of shape: [Batch, H, W ,C].
    Args:
        n_iter: number of iterations for MCT to calibrate on
    Returns:
        A representative dataset generator
    """       
    def representative_dataset() -> Iterator[List]:
        ds_iter = iter(dataset_loader)
        for _ in range(n_iter):
            yield [next(ds_iter)[0]]

    return representative_dataset

# Get representative dataset generator
representative_dataset_gen = get_representative_dataset(n_iter=n_iters, dataset_loader=val_loader)

# Perform post training quantization with the default configuration
quant_model, _ = mct.ptq.pytorch_post_training_quantization(model, representative_data_gen=representative_dataset_gen)
print('Quantized model is ready')

Statistics Collection: 20it [00:10,  1.86it/s]



Running quantization parameters search. This process might take some time, depending on the model size and the selected quantization methods.



Calculating quantization parameters: 100%|██████████| 102/102 [00:17<00:00,  5.92it/s]


Weights_memory: 3469760.0, Activation_memory: 1204224.0, Total_memory: 4673984.0, BOPS: 4999000429690880

Please run your accuracy evaluation on the exported quantized model to verify it's accuracy.
Checkout the FAQ and Troubleshooting pages for resolving common issues and improving the quantized model accuracy:
FAQ: https://github.com/sony/model_optimization/tree/main/FAQ.md
Quantization Troubleshooting: https://github.com/sony/model_optimization/tree/main/quantization_troubleshooting.md
Quantized model is ready


### Model Export

Now, we can export the quantized model, ready for deployment, into a `.onnx` format file. Please ensure that the `save_model_path` has been set correctly. 

In [19]:
mct.exporter.pytorch_export_model(model=quant_model,
                                  save_model_path='./torchvision_qmodel.onnx',
                                  repr_dataset=representative_dataset_gen,
                                  onnx_opset_version=17)

Exporting onnx model with MCTQ quantizers: ./torchvision_qmodel.onnx


## Evaluation on ImageNet dataset

### Floating point model evaluation
Please ensure that the dataset path has been set correctly before running this code cell.

In [ ]:
from tutorials.resources.utils.pytorch_tutorial_tools import classification_eval
# Evaluate the model on ImageNet
eval_results = classification_eval(model, val_loader)

# Print float model Accuracy results
print("Float model Accuracy: {:.4f}".format(round(100 * eval_results[0], 2)))

Classification evaluation:   2%|▏         | 262/12500 [00:03<02:21, 86.72it/s]

Num of images: 1000, Accuracy: 85.5 %


Classification evaluation:   4%|▍         | 515/12500 [00:06<02:11, 91.19it/s]

Num of images: 2000, Accuracy: 81.5 %


Classification evaluation:   6%|▌         | 765/12500 [00:09<02:17, 85.07it/s] 

...

Classification evaluation:  12%|█▏        | 1512/12500 [00:18<02:20, 78.14it/s] 

Num of images: 6000, Accuracy: 78.87 %


Classification evaluation:  14%|█▍        | 1766/12500 [00:21<02:02, 87.58it/s]

Num of images: 7000, Accuracy: 79.49 %


Classification evaluation:  15%|█▍        | 1846/12500 [00:22<01:56, 91.15it/s]

### Quantized model evaluation
We can evaluate the performance of the quantized model. There is a slight decrease in performance that can be further mitigated by either expanding the representative dataset or employing MCT's advanced quantization methods, such as GPTQ (Gradient-Based/Enhanced Post Training Quantization).

In [ ]:
# Evaluate the quantized model on ImageNet
eval_results = classification_eval(quant_model, val_loader)

# Print quantized model Accuracy results
print("Quantized model Accuracy: {:.4f}".format(round(100 * eval_results[0], 2)))

\
Copyright 2024 Sony Semiconductor Israel, Inc. All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License